In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the data from CSV instead of MongoDB
df = pd.read_csv('aac_shelter_outcomes.csv')

# Drop the '_id' column if it exists (to match MongoDB structure)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Dashboard Layout / View
app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo
image_filename = '/home/jameshatzibir_snhu/Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Layout of the dashboard
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
        style={'textAlign': 'center'}
    ),
    html.Center(html.B(html.P('Unique Identifier: JJH340'))),  # Add this line
    html.Hr(),
    html.Label('Animal Type:'),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Dog', 'value': 'Dog'},
            {'label': 'Cat', 'value': 'Cat'}
        ],
        value='All',
        labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        row_deletable=False,
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        style_cell={'textAlign': 'left', 'overflow': 'hidden', 'textOverflow': 'ellipsis', 'maxWidth': 0},
        style_cell_conditional=[
            {'if': {'column_id': 'name'},
             'width': '20%'},
            {'if': {'column_id': 'breed'},
             'width': '30%'},
        ],
        style_data_conditional=[
            {'if': {'row_index': 'odd'},
             'backgroundColor': 'rgb(248, 248, 248)'}
        ],
        style_header={'backgroundColor': 'rgb(230, 230, 230)',
                      'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

# Callback to filter data table based on animal type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'All':
        data = df.to_dict('records')
    else:
        data = df[df['animal_type'] == filter_type].to_dict('records')
    return data

# Callback to update the pie chart based on the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available to generate chart.")]
    
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    
    return [dcc.Graph(figure=fig)]

# Callback to highlight selected row in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'backgroundColor': '#D2F3FF'
    } for i in selected_columns]

# Callback to update the map based on selected row in the data table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None or len(viewData) == 0:
        return [html.P("No data available to display on the map.")]

    dff = pd.DataFrame.from_dict(viewData)

    if not dff.empty and 'location_lat' in dff.columns and 'location_long' in dff.columns:
        row = index[0] if index else 0
        
        try:
            lat = float(dff.iloc[row]['location_lat'])
            lon = float(dff.iloc[row]['location_long'])

            # Check if the latitude and longitude values are within valid ranges
            if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
                raise ValueError("Invalid lat/lon values.")
            
        except (ValueError, TypeError, KeyError):
            return [html.P("Invalid location data; cannot display on the map.")]

        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(dff.iloc[row]['breed']),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row]['name'])
                    ])
                ])
            ])
        ]
    else:
        return [html.P("No data available to display on the map.")]

# Run app and display result inline in the notebook
app.run_server(mode='external', port=8050, debug=True)

/home/jameshatzibir_snhu/.local/lib/python3.9/site-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/
